# **Dictionary and Sets - Properties**

Dictionaries and Sets are great to hold data which dont have a inherent order and unique. Another important characteristic is they should be hashable. Due to this hashable property we are able to access/search items in a list with O(1) time complexity. Also their insertion time is O(1)(depends on the hash function).

But those compelling properties comes with a cost like additional memory footprint, additional complexiy/time cost for other operations etc.

Dictionaries and sets work using hash functions and tables. There we first define a memory area and using the Hash function we identify where we need to put the element(index). (Knowledge about hash collisions and all the other craps related to hasp maps will be needed!)


But basically what happens is when we try to insert a value to dictionary, first we calculate the hash and then we mask it. This effectively turn the hash value (very large value) to a index in an array (or put the value in to one of the predefined buckets)

* eg:- 
    - hash value = 28794
    - num of buckets = 8
    - then index = 28794 & 0b111 = 2 so the second index.

The mask value of this case is 0b111 since we only have 8 memoy blocks. But if we have more then we need to make the mask resized as well.

Then we need to check the bucket/index is already in use. If thats the case we can check the current value with existing value. If they are same, then no issue. Else we need to find a new place to add.

> Apparently in python instead of storing values in predefined buckets it saves item index in a array. Then use it to access the actual data in the memory. This is a optimization used by python. Because of this property in python dictionary we can keep track of in which order items were added!

To find a new index in case of hash collision, python uses a linear function `probing`. Here we use another mask using the item's higher order hash values. Psuedo implementation is as followed.

In [14]:
def index_sequence(key, mask=0b111, collision_shift=5):
    hash_val = hash(key) # This returns a integer.
    i = hash_val & mask
    yield i

    # If theres a collision this part will help. (very cleaver IMO!)
    while True:
        hash_val >>= collision_shift
        i = (i * 5 + hash_val + 1) & mask
        yield i

> The while(True) in the code helps to avoid continuous hash collisions for hash values which have same bits for corresponding mask part bits. Also when defining a hash function for data types it is important to design them to reduce hash collisions as much as possible.

We do the exact same thing as above for lookups as well.

But when it comes to deletions things are bit different. We cant simply mark the deleted item location as null. In insertion process we check for null values to solve hash collisions. Therefore we need to make sure deletion does not affect that. To do that we write a special value to mark the deleted item.

In python, dictionary, hash table must be dynamically resized to accomadate to the new items. This is an expensive process as all the already existing elements need to be reinserted with new mask values. Since this does not happen often still the amortized time complexity is considered as O(1). Expanding algorithm is as follows.

Always start with 8 slots.
If 2/3 slots are full, then size need to be increased by 3 times.

    eg:- after the sixth element, memory slots will get increased to 18. From there once 13th item inserted slots will be increased to 39 etc.
(Also resizing can happen to reduce size as well)


> In python if we need to make a custom class hashable, we can either use default hash which is the memory location id. Or we can define a custom __hash__ function with custom __eq__ function so that comparisons can be made.

</br>

> Also it important to define the hash function to return evenly distributed values. This property ensures the minimum number of collisions between items.


In [3]:
import string
import timeit


class BadHash(str):

    def __hash__(self) -> int:
        return 42

class GoodHash(str):

    def __hash__(self) -> int:
        a_ord = ord('a')
        return (ord(self[1]) - a_ord)+(ord(self[0]) - a_ord)


In [11]:
good_dict = set()
bad_dict = set()


for let1 in string.ascii_lowercase:
    for let2 in string.ascii_lowercase:
        key = let1+let2

        good_dict.add(GoodHash(key))
        bad_dict.add(BadHash(key))

In [12]:
%timeit GoodHash('zz') in good_dict

354 ns ± 2.6 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [13]:
%timeit BadHash('zz') in bad_dict

10.8 µs ± 147 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


We can see the effect of having good and bad hashing function using above examples. Below is the same experiment but with different timeit module usage pattern.

In [18]:
timeit.repeat(
"key in bad_dict",
 '''from __main__ import bad_dict, BadHash
key = BadHash('zz')''',
 repeat=3, 
 number=1_000_000)

[10.612844799999948, 10.61202030000004, 10.67579749999959]

In [19]:
timeit.repeat(
"key in good_dict",
 '''from __main__ import good_dict, GoodHash
key = GoodHash('zz')''',
 repeat=3, 
 number=1_000_000)

[0.21189019999974334, 0.20485240000016347, 0.21517530000028273]

Another Fact about python is the way it looks for variables in names spaces. 
It normally flows like below

1. locals() array - stores local variables
2. globals() dictionary - stores global level variables
3. \_\_builtin\_\_ object - technically module object which stores module level values. (here we check its locals() map for our required value)

In [27]:
import math
from math import sin


def test1(x):
    res = 1
    for _ in range(1000):
        res+= math.sin(x)  #Accesing the function from module level __builtin__ object
    return res

def test2(x):
    res = 1
    for _ in range(1000):
        res+= sin(x)  # Function is already loaded from module, and now in globals().
    return res

def test3(x, func=math.sin):
    res = 1
    for _ in range(1000):
        res+= func(x)  # accessing the function as a local variable.
    return res

In [28]:
%timeit test1(90)

115 µs ± 307 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
%timeit test2(90)

100 µs ± 450 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [30]:
%timeit test3(90)

94.4 µs ± 575 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [32]:
import dis

In [33]:
dis.dis(test1)

  6           0 LOAD_CONST               1 (1)
              2 STORE_FAST               1 (res)

  7           4 LOAD_GLOBAL              0 (range)
              6 LOAD_CONST               2 (1000)
              8 CALL_FUNCTION            1
             10 GET_ITER
        >>   12 FOR_ITER                18 (to 32)
             14 STORE_FAST               2 (_)

  8          16 LOAD_FAST                1 (res)
             18 LOAD_GLOBAL              1 (math)
             20 LOAD_METHOD              2 (sin)
             22 LOAD_FAST                0 (x)
             24 CALL_METHOD              1
             26 INPLACE_ADD
             28 STORE_FAST               1 (res)
             30 JUMP_ABSOLUTE           12

  9     >>   32 LOAD_FAST                1 (res)
             34 RETURN_VALUE


In [34]:
dis.dis(test2)

 12           0 LOAD_CONST               1 (1)
              2 STORE_FAST               1 (res)

 13           4 LOAD_GLOBAL              0 (range)
              6 LOAD_CONST               2 (1000)
              8 CALL_FUNCTION            1
             10 GET_ITER
        >>   12 FOR_ITER                16 (to 30)
             14 STORE_FAST               2 (_)

 14          16 LOAD_FAST                1 (res)
             18 LOAD_GLOBAL              1 (sin)
             20 LOAD_FAST                0 (x)
             22 CALL_FUNCTION            1
             24 INPLACE_ADD
             26 STORE_FAST               1 (res)
             28 JUMP_ABSOLUTE           12

 15     >>   30 LOAD_FAST                1 (res)
             32 RETURN_VALUE


In [35]:
dis.dis(test3)

 18           0 LOAD_CONST               1 (1)
              2 STORE_FAST               2 (res)

 19           4 LOAD_GLOBAL              0 (range)
              6 LOAD_CONST               2 (1000)
              8 CALL_FUNCTION            1
             10 GET_ITER
        >>   12 FOR_ITER                16 (to 30)
             14 STORE_FAST               3 (_)

 20          16 LOAD_FAST                2 (res)
             18 LOAD_FAST                1 (func)
             20 LOAD_FAST                0 (x)
             22 CALL_FUNCTION            1
             24 INPLACE_ADD
             26 STORE_FAST               2 (res)
             28 JUMP_ABSOLUTE           12

 21     >>   30 LOAD_FAST                2 (res)
             32 RETURN_VALUE


As you can see 3 functions have different access patterns to the `sin` function.